# AUTOMATIC GENERATION OF MDA ANALYSIS FROM REQUIREMENTS

**First step is to search the Papyrus Requirements model:**

In [1]:
def uml_xml_json_parse():
    import shutil
    import os

#Make no attention to the "r" character before the directory path's examples. It is only a python syntax so the "\" character can be taken literally.

    dir_or = input("Please enter the papyrus model directory path:\n")
# During the developement period the path used was the one from the Papyrus workspace. 
# See examples below:
# r'C:\Users\rroja\workspace-papyrus\Sellar\Sellar.uml'
# C:\Users\rroja\Documents\ISAE\Research Project\Papyrus_models\Common Requirements MBSA\RP1_seq1.uml
    dir_target = input("Please crate a directory path to parse the original model file and thus avoid any damage to the original source code.\n"+
                   "Once created please input it below:\n (Please notice that results from OpenMDAO will be stored in a folder created automatically in this directory)\n")
#As dir_target the path used was as below. It can be any preferred location, example below.
#dir_target = r'C:\Users\rroja\Documents\ISAE\Research Project\Jupyter_Tests\Copied_uml-to-xml'

#The name of the generated copy is with a direct parse to XML file. model_XML.xml
    name='model_XML.xml'
    path = os.path.join(dir_target, name)

    shutil.copyfile(dir_or, path)
    print('xml file parsed')
    
    import json
    import xmltodict

    with open(r''+path+'') as xml_file:
        data_dict = xmltodict.parse(xml_file.read())
        xml_file.close()

    json_data = json.dumps(data_dict)
    with open("data.json", "w") as json_file:
        json_file.write(json_data)
        json_file.close()
        
    # JSON file
    
    f = open ('data.json', "r")
  
    # Reading from file
    data = json.loads(f.read())
    
    return data

In [2]:
json_data = uml_xml_json_parse()

Please enter the papyrus model directory path:
C:\Users\rroja\Documents\ISAE\Research Project\Papyrus_models\Common Requirements MBSA\RP1_seq1.uml
Please crate a directory path to parse the original model file and thus avoid any damage to the original source code.
Once created please input it below:
 (Please notice that results from OpenMDAO will be stored in a folder created automatically in this directory)
C:\Users\rroja\Documents\ISAE\Research Project\Jupyter_Tests\Copied_uml-to-xml
xml file parsed


**A dictionary with requirements' xmi:id as keys and id as values:**

It is performend for all requirements so we can later retrieve the actual requirements id liked to MDAO analysis and the corresponding quantity of interest (QoI).

The origin of this step comes from the fact the a requirement name might be different than the given requirement id or variable. Therefore, the Papyrus model' source code gives an indentifier to link each block to its attributes. Thus, the algorithm serves of it to automatically retrieve the wanted information (variable Id towards MDAO and QoI).

In [3]:
#json_data

In [4]:
#Function creating the parameters diagram directly, not dependant on model's complexity.
def MDAO_ids_QoI(json_data):
    Parameters_dict={}
    for Requirements in json_data["xmi:XMI"]["Requirementprofile:RequirementPlus"]:
        if Requirements.get("QoI")==None or Requirements.get("@MDAO_id")==None :
            Parameters_dict[Requirements.get("@MDAO_id")]=Requirements.get("QoI")
    
    return Parameters_dict

In [10]:
#Function to troubleshoot the data from a standalone requirements diagram.
def MDAO_ids_QoI_TS(json_data):
#Constructing the dictionary that gatter the Ids liked to MDAO with the internal papyrus @xmi:id.
    id_dict={}

    for Requirements in json_data["xmi:XMI"]["Requirementprofile:RequirementPlus"]:
        id_dict[Requirements.get("@base_NamedElement")]=Requirements.get("@MDAO_id")    

    id_dict

    #Constructing the names vs xmi:id dictionary
    name_dict={}
    for req_block in json_data["xmi:XMI"]["uml:Model"]['packagedElement']:
        if req_block.get('@xmi:type')=='uml:Package':
            if type(req_block['packagedElement']) == dict:
                name_dict[req_block['packagedElement']['@xmi:id']]=req_block['packagedElement']['@name']
            else:
                for variable in req_block['packagedElement']:
                    name_dict[variable.get('@xmi:id')]=variable.get("@name")


    #Code to automatically create the requirements dictionary (id:name)
    #The aim is to verify that the correlation between the information is correct.
    Requirements_dict = {}
    for xmi_id in id_dict:
        if xmi_id in name_dict.keys():
             Requirements_dict[id_dict[xmi_id]]=name_dict[xmi_id]

    #WE SHOULD ADDRESS ONLY THE NEEDED PARAMETERS FROM THE MDAO PROBLEM.
    #below we get all of them that have a "QoI" just to code the case when a req. has not this attribute.

    Parameters_dict = {}
    for Requirements in json_data["xmi:XMI"]["Requirementprofile:RequirementPlus"]:    
        if Requirements.get("QoI")==None:
            pass
        else:
            if Requirements.get("@MDAO_id") in Requirements_dict.keys():
                Parameters_dict[Requirements.get("@MDAO_id")] = Requirements["QoI"]

    return Parameters_dict

In [5]:
Parameters_dict=MDAO_ids_QoI(json_data)

**WhatsOpt log in:**

In [6]:
    # Completely necessary
!pip install -U wop

from whatsopt.whatsopt_client import WhatsOpt
wop = WhatsOpt(url="https://ether.onera.fr/whatsopt")
ok = wop.login(echo=True)

!wop status
#!wop list

Requirement already up-to-date: wop in c:\users\rroja\anaconda3\lib\site-packages (2.0.0)
Successfully logged into WhatsOpt (https://ether.onera.fr/whatsopt)

You are logged in https://ether.onera.fr/whatsopt
No local analysis found
  (use 'wop list' and 'wop pull <id>' to retrieve an existing analysis)
  (use 'wop push <analysis.py>' to push from the local OpenMDAO code to the server)



**Pull of the project storing all the disciplines:**

In [7]:
def Project_Json_Pull(pid):
    import subprocess
    
    cmd_Ppull ="wop pull --json --project-id " + str(pid)
    
    Project_Pull = subprocess.run(cmd_Ppull,capture_output=True, text=True).stdout

    #print(Project_Pull)
    #Project_Pull_str=Project_Pull[0]

    with open("Project_Pull_def.json", "w") as json_file:
        json_file.write(Project_Pull)
        json_file.close()

    import json
    # Disciplines JSON file
    p = open ('Project_Pull_def.json', "r")
    pulled_project = json.loads(p.read())

    return pulled_project

In [8]:
P14 = Project_Json_Pull(14)

In [9]:
P14

{'name': 'UAV_tools_test',
 'created_at': '2022-02-16T16:45:21.813Z',
 'owner_email': 'rafael.rojas-cardenas-@student.isae-supaero.fr',
 'description': 'Application of methodology on MDA for UAV',
 'analyses_attributes': [{'name': 'Masses_Analysis',
   'disciplines_attributes': [{'name': '__DRIVER__',
     'type': 'null_driver',
     'variables_attributes': [{'name': 'Empennage_weight',
       'io_mode': 'out',
       'shape': '1',
       'type': 'Float',
       'desc': '',
       'units': '',
       'active': True,
       'distributions_attributes': []},
      {'name': 'Endurance',
       'io_mode': 'out',
       'shape': '1',
       'type': 'Float',
       'desc': '',
       'units': '',
       'active': True,
       'distributions_attributes': []},
      {'name': 'Fuselage_weight',
       'io_mode': 'out',
       'shape': '1',
       'type': 'Float',
       'desc': '',
       'units': '',
       'active': True,
       'distributions_attributes': []},
      {'name': 'MTOW',
       'i

**Generation of a list containing all the disciplines from the project:**

In [10]:
def Project_Disc_List(pulled_project):
    #Creation of the list for all the sub-analysis (Disciplines) in the whatsopt project
    Project_Disc_list = []

    #Disciplines parse
    for disciplines in pulled_project['analyses_attributes']:
        Project_Disc_list.append(disciplines)

    return Project_Disc_list

In [11]:
PDL14 = Project_Disc_List(P14)

**Generation of a list of all the disciplines whose outputs are found in the requirements:**

In [12]:
def Disciplines_in_MBSE(Project_Disc_list,Parameters_dict):
    #Discipline List append algorithm for the outputs variables that are found in the req. diagram.
    MDA_Disc_list = []

    for Disc in Project_Disc_list:
        for disc_attr in Disc["disciplines_attributes"]:
            if disc_attr['name']+'_Analysis' == Disc['name']:
                for var_attr in disc_attr['variables_attributes']:
                    if var_attr['name'] in Parameters_dict.keys() and var_attr['io_mode'] == 'out':
                        if disc_attr['name'] not in MDA_Disc_list:
                            MDA_Disc_list.append(Disc)
    

    return MDA_Disc_list


In [13]:
MDA_DL14=Disciplines_in_MBSE(PDL14,Parameters_dict)

**Generation of the MDA json file to be pushed:
First a list of all the variables of the analysis**

In [14]:
def MDA_json_to_push(MDA_Disc_list,Parameters_dict,Project_Disc_list):

    #Algorithm to find the disciplines whose outputs are inputs for the disciplines previoulsy found.
    #This algorithm is a loop that finishes when there are no more disciplines to be added to the list.
    
    import json
    Inputs_list = []

    while(True):

        MDA_list_size_init= len(MDA_Disc_list)

        for Disc in MDA_Disc_list:
            for disc_attr in Disc["disciplines_attributes"]:
                if disc_attr['name']+'_Analysis' == Disc['name']:
                    for var_attr in disc_attr['variables_attributes']:
                        if var_attr['io_mode'] == 'in':
                            if var_attr['name'] not in Inputs_list:
                                Inputs_list.append(var_attr['name'])

    #Inputs_list

        for Disc in Project_Disc_list:
            for disc_attr in Disc["disciplines_attributes"]:
                if disc_attr['name']+'_Analysis' == Disc['name']:
                    for var_attr in disc_attr['variables_attributes']:
                        if var_attr['io_mode'] == 'out':
                            if (var_attr['name'] in Inputs_list) and (Disc not in MDA_Disc_list):
                                MDA_Disc_list.insert(0,Disc)

        MDA_list_size_up= len(MDA_Disc_list)

        if MDA_list_size_up != MDA_list_size_init:
            MDA_list_size_init = MDA_list_size_up
            continue
        else:
            break    
            
    list_of_variables=[]

    #DRONE_MDA concatenation.

    #FOR LOOP TO RETRIEVE ALL THE DISCIPLINES THAT MIGHT HAVE AN output AS input for our original disc.

    #we start to append all "out" variables in the list without any change.
    for Disc in MDA_Disc_list:
        for disc_attr in Disc["disciplines_attributes"]:
            if disc_attr['name']+'_Analysis' == Disc['name']:
                for var_attr in disc_attr['variables_attributes']:
                    if var_attr['name'] in [v_dict['name'] for v_dict in list_of_variables]:
                        pass
                    elif var_attr['io_mode'] == 'out':
                        list_of_variables.append(var_attr)
   
    # Now we have to search all the "in" variables for each discipline leaving outside all previous "out" variables.


    for Disc in MDA_Disc_list:
        for disc_attr in Disc["disciplines_attributes"]:
            if disc_attr['name']+'_Analysis' == Disc['name']:
                for var_attr in disc_attr['variables_attributes']:
                    if var_attr['name'] in [v_dict['name'] for v_dict in list_of_variables]:
                        pass
                    else:
                        Inputs_list.append(var_attr)
                        list_of_variables.append(var_attr)

#The QoI is inserted to those variables that are found in the Parameters_dict.

    #Algorithm to modify the top level DRIVER variables initial value.
     
    init_value_template = {'init': None, 'lower': None, 'upper': None}


    for var in list_of_variables:
        if var['name'] in Parameters_dict.keys():
            init_value_template['init']=Parameters_dict.get(var['name'])
            var['parameter_attributes']=init_value_template
            init_value_template = {'init': None, 'lower': None, 'upper': None}

    #Algorithm to modify the local discipline Driver.

    init_value_template = {'init': None, 'lower': None, 'upper': None}

    for Disc in Project_Disc_list:
        for disc_attr in Disc["disciplines_attributes"]:
            if disc_attr['name'] == '__DRIVER__':
                for var_attr in disc_attr['variables_attributes']:
                    if var_attr['name'] in Parameters_dict.keys():
                        init_value_template['init']=Parameters_dict.get(var_attr['name'])
                        var_attr['parameter_attributes']=init_value_template
                        init_value_template = {'init': None, 'lower': None, 'upper': None}

    #if the variables are out in a discipline then they shall be "in" at the driver.
    # inversely, if the variables are an independent "in", then they are "out" from the driver.
                        
    #For the Driver, the io_mode needs to be switched:
    #Notice that a "new_list" is needed in order to make a "deep copy" and do not modify the original format."   
    import copy
    new_list = copy.deepcopy(list_of_variables)
    for instance in new_list:
        if instance['io_mode'] == 'out':
            instance['io_mode'] = 'in'
        elif instance['io_mode'] == 'in':
            instance['io_mode'] = 'out'
    #Insert into the json file the list of variables:
    #First we start to generate the format for the DRIVER
    
    DRONE_MDA_push = {'name': 'DRONE_Reference_push', 
             'disciplines_attributes': [{
                 'name': '__DRIVER__', 
                 'type': 'null_driver',
                 'variables_attributes':[]}]}

    DRONE_MDA_push['disciplines_attributes'][0]['variables_attributes'] = new_list
    
    #Insert the disciplines:
    for disc in MDA_Disc_list:
        DRONE_MDA_push['disciplines_attributes'].append({'name': disc['name'],'type': 'mda', 'sub_analysis_attributes': disc})

    #Convert json file into a string so as to be pushed to WhatsOpt:
    string_problem_mod = json.dumps(DRONE_MDA_push)
        
    return string_problem_mod

In [15]:
json_to_push=MDA_json_to_push(MDA_DL14,Parameters_dict,PDL14)

In [16]:
json_to_push

'{"name": "DRONE_Reference_push", "disciplines_attributes": [{"name": "__DRIVER__", "type": "null_driver", "variables_attributes": [{"name": "Fuselage_area", "io_mode": "in", "shape": "1", "type": "Float", "desc": "", "units": "", "active": true, "distributions_attributes": []}, {"name": "Fuselage_weight", "io_mode": "in", "shape": "1", "type": "Float", "desc": "", "units": "", "active": true, "distributions_attributes": []}, {"name": "L_HT", "io_mode": "in", "shape": "1", "type": "Float", "desc": "", "units": "", "active": true, "distributions_attributes": []}, {"name": "L_VT", "io_mode": "in", "shape": "1", "type": "Float", "desc": "", "units": "", "active": true, "distributions_attributes": []}, {"name": "Chord", "io_mode": "in", "shape": "1", "type": "Float", "desc": "", "units": "", "active": true, "distributions_attributes": []}, {"name": "Cx", "io_mode": "in", "shape": "1", "type": "Float", "desc": "", "units": "", "active": true, "distributions_attributes": []}, {"name": "Cz", 

**Push into WhatsOpt:**

In [17]:
    #Parsing json into text to push into whatsopt

with open("problem_definition_mod.json", "w") as json_file:
        json_file.write(json_to_push)
        json_file.close()

!wop push --json problem_definition_mod.json

Analysis DRONE_Reference_push pushed


**Pulling the project to run it**

In [95]:
#Getting the last modified project id
owned_project_list = !wop list
last_project=owned_project_list[len(owned_project_list)-2]
lp_list=last_project.split(' ')
lp_id=lp_list[0]
lp_id

'1633'

In [96]:
#Pulling the last modified project
newpath = r''+dir_target+''+'\OpenMDAO_Files\\'+lp_id
if not os.path.exists(newpath):
    os.makedirs(newpath)
os.chdir(newpath)
!dir

 El volumen de la unidad C es Windows
 El número de serie del volumen es: DAFC-E9CF

 Directorio de C:\Users\rroja\Documents\ISAE\Research Project\Jupyter_Tests\Copied_uml-to-xml\OpenMDAO_Files\1633

09/02/2022  10:11 a. m.    <DIR>          .
09/02/2022  10:11 a. m.    <DIR>          ..
               0 archivos              0 bytes
               2 dirs  22,719,012,864 bytes libres


In [97]:
#How to automatically pull the lastest modification of the project.

cmd_lp = "wop pull " + lp_id

import subprocess
subprocess.run(cmd_lp, shell=True, check=True)

CompletedProcess(args='wop pull 1633', returncode=0)

In [98]:
%run run_mda.py

|  
|  ==============
|  disc1_analysis
|  ==============
|  NL: NLBGS Converged in 1 iterations
|  
|  ==============
|  disc2_analysis
|  ==============
|  NL: NLBGS Converged in 1 iterations
NL: NLBGS Converged in 1 iterations
8 Input(s) in 'model'

varname         val 
--------------  ----
disc1_analysis
  disc1
    x           [1.]
    y2          [1.]
    z1          [1.]
    z2          [1.]
disc2_analysis
  disc2
    m1          [1.]
    y1          [1.]
    z1          [1.]
    z2          [1.]


2 Explicit Output(s) in 'model'

varname         val 
--------------  ----
disc1_analysis
  disc1
    y1          [1.]
disc2_analysis
  disc2
    y2          [1.]


0 Implicit Output(s) in 'model'




In [99]:
#!wop -h